<a href="https://colab.research.google.com/github/karenbennis/Xy/blob/ml_model/pyspark_pipeline_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>**Connect to Database**<br><br>

In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

#Interact with SQL
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Start Spark Session(Creating spark application with name defined by appName()) ---IMPORTED WITH EVERY COLAB NOTEBOOK
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("database_transformation").config("spark.driver.memory","8g").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


--2020-07-25 18:02:45--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-07-25 18:02:47 (1.05 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
# gcloud login and check the DB
!gcloud auth login
!gcloud config set project 'xy-yelp'
!gcloud sql instances describe 'xy-yelp'

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=TGHQzzth5e5AMSwqAw6CpiH3wec9voSu-hui-MK-he0&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account


Enter verification code: 4/2QHe7YUZLypxZoQQ6PhcY2igPX1F0aY8znLdi8Pv6YIxOPyVrLPM_vU

You are now logged in as [helenly25@gmail.com].
Your current project is [xy-yelp].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
backendType: SECOND_GEN
connectionName: xy-yelp:northamerica-northeast1:xy-yelp
databaseV

In [ ]:
# download and initialize the psql proxy
!wget https://dl.google.com/cloudsql/cloud_sql_proxy.linux.amd64 -O cloud_sql_proxy
!chmod +x cloud_sql_proxy
# "connectionName" is from the previous block
!nohup ./cloud_sql_proxy -instances="xy-yelp:northamerica-northeast1:xy-yelp"=tcp:5432 &
!sleep 30s

cloud_sql_proxy: Text file busy
nohup: appending output to 'nohup.out'


In [ ]:
db_password = 'kjhbyelpdb'

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://127.0.0.1:5432/xy_yelp_db"
config = {"user":"postgres", 
          "password": db_password, 
          "driver":"org.postgresql.Driver"}

**Extract tables**

In [ ]:
# Pull review table
review_df2 = spark.read.jdbc(url=jdbc_url, table='review',properties=config)
review_df2.show(5)

+--------------------+--------------------+-----+----+------+-----+-----------+-----------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|review_type|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+
|cALYebKb5hygdKHql...|This is a very in...|    4|   0|     0|    0| 2011-01-12|     review|
|SawdMXLYD5ytRmMFv...|I LOVE Chic Nails...|    5|   0|     2|    0| 2011-01-20|     review|
|j-jMQdELr6AFkCcEH...|After the Padres ...|    5|   0|     0|    0| 2011-01-06|     review|
|SmUMyCUNrT9HEo_DX...|I have to admit t...|    4|   0|     1|    0| 2010-01-17|     review|
|oTB_mpCKcu-8wayQQ...|Best food, super ...|    5|   0|     1|    0| 2011-01-14|     review|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+
only showing top 5 rows



In [ ]:
# Pull business table
business_df2 = spark.read.jdbc(url=jdbc_url, table='business',properties=config)
business_df2.show(5)

+--------------------+--------------------+
|           review_id|         business_id|
+--------------------+--------------------+
|fWKvX83p0-ka4JS3d...|9yKzy9PApeiPPOUJE...|
|IjZ33sJrzXqU-0X6U...|ZRJwVLyzEJq1VAihD...|
|IESLBzqUCLdSzSqm0...|6oRAC4uyJCsJl1X0W...|
|G-WvGaISbqqaMHlNn...|_1QQZuf4zZOyFCvXc...|
|1uJFq2r5QfJG_6ExM...|6ozycU1RpktNG2-1B...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Pull yelp_user table
user_df2 = spark.read.jdbc(url=jdbc_url, table='yelp_user',properties=config)
user_df2.show(5)

+--------------------+--------------------+
|           review_id|             user_id|
+--------------------+--------------------+
|GJGUHAAONtBSBj53c...|Z3c7xGRfeV-uMkSbA...|
|nQH2KAvAeOJOYKX99...|ryjqXdp68i2I9JPOp...|
|-yKcbjWSlmKC1zTMT...|5W-ruHmpkwLyI6Lla...|
|20aES_-g5Vyqfzojn...|vhxFLqRok6r-D_aQz...|
|W_d9w7yr3koSUXHco...|aBnKTxZzdhabTXfzt...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
# Join tables
spark_df = review_df2.join(business_df2, on="review_id", how="inner")
spark_df = spark_df.join(user_df2, on="review_id", how="inner")
spark_df.show(5)

+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|review_type|         business_id|             user_id|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|     review|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|
|-Be0UUGYuiDJVAM_Y...|Since Im big into...|    4|   0|     2|    2| 2011-01-25|     review|pa6K7DGByxBXxcVJ5...|_4lqpCYCqOQzbB6xQ...|
|-nQHHXi-d_yuW301_...|A pleasant place ...|    2|   0|     0|    0| 2011-01-12|     review|GIGI8bJfN6HyPzmEW...|4QORbyhfN01oKR_Gg...|
|2L30O7G8IQ6HILpR0...|part of a social ...|    5|   0|     0|    0| 2010-01-24|     review|qiwajZigq_2twTmYo...|ST8Yzlk2MqKlcaLqL...|
|4x5yLG7_yGLuN-w6f...|I love every plac...|    4|   0|     1| 

**Transformation**

In [ ]:
import pyspark.sql.functions as F

spark_df=spark_df.withColumn('length',F.length('review_text'))

In [ ]:
spark_df = spark_df.filter((spark_df.stars != 2) & (spark_df.stars != 4))
spark_df.show()

+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+------+
|           review_id|         review_text|stars|cool|useful|funny|review_date|review_type|         business_id|             user_id|length|
+--------------------+--------------------+-----+----+------+-----+-----------+-----------+--------------------+--------------------+------+
|-7yxrdY13ay15rGB7...|I have been going...|    5|   0|     0|    0| 2010-01-16|     review|Lh9nz0KYyzE-YRbKu...|ayKW9eWwGFcrtJaHc...|   670|
|2L30O7G8IQ6HILpR0...|part of a social ...|    5|   0|     0|    0| 2010-01-24|     review|qiwajZigq_2twTmYo...|ST8Yzlk2MqKlcaLqL...|   415|
|5h0EVAee-RDbbKfhd...|A great value for...|    5|   2|     1|    0| 2012-01-21|     review|4VzaYvZntWBRbr8Vm...|feQpvbp8jGBWMuG5u...|   284|
|6jV77Bs_Vu_rHkdUx...|This review is fo...|    3|   1|     2|    1| 2011-01-23|     review|kwq3bK7BzPKLwXKqV...|RwVaQNP1Ag-Seu3U9...|  1092|
|8uV26l7-ktb4

In [ ]:
# spark_df=spark_df.withColumn('class',F.when( (spark_df["stars"]==1), 0).when((spark_df["stars"]))
# spark_df.show()

In [2]:
# convert to pandas
import pandas as pd
pandas_df = pd.read_csv("https://raw.githubusercontent.com/karenbennis/Xy/storyboard/uniform_yelp.csv")

# Set index
# pandas_df = pandas_df.set_index('review_id')

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,K8avYPWsh45v7VoZgQz-Vg,mrEoKZsYtSgMJsIuLr_oTg,UGyEr_PMA-v1cuim0gMPlQ,3,2008-11-23,another pie place that I would be happy to mee...,0,0,0
1,BkiZn5XSzAv9q7J7_4HP7A,7HFkF9A2xaHxLqwcFnJ0JQ,N_2yEZ41g9zDW_gWArFiHw,4,2017-03-30,I came with my sister and was very excited fro...,1,0,1
2,L6kc7Nr7hWiqo7ZvWu4rtg,rsi9Sf_Efk55hPRk6k7vpQ,XhLM_OtYslzyd4GyvpqACA,1,2017-03-19,I am very disappointed with the Brazilian I re...,0,0,0
3,y35xKzutHXT985mUpp5qVA,fKSX1CAoXkyQecAWR02DVQ,ILa-Xv5-h23A9OMrYK17oQ,4,2017-05-04,Stopped for lunch with my wife. I had the Mont...,0,0,1
4,UqQGtBDEfkYMLV-FykNCdA,YmTYfzrC4Z2JbyOucGVJtA,gBfPyzPRmeOaj3SdcIj0Rw,1,2010-05-20,"DON'T DO IT! You're in Vegas, looking for a d...",1,0,0


In [3]:
# Import dependencies for nltk
# https://towardsdatascience.com/natural-language-processing-nlp-for-machine-learning-d44498845d5b
import nltk

In [4]:
# Import string and punctuations
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
# Function to remove Punctuation
def remove_punct(text):

  # Discard all punctuations
  text_nopunct = ''.join([char for char in text if char not in string.punctuation])
  return text_nopunct

pandas_df['body_text_clean'] = pandas_df['text'].apply(lambda x: remove_punct(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean
0,K8avYPWsh45v7VoZgQz-Vg,mrEoKZsYtSgMJsIuLr_oTg,UGyEr_PMA-v1cuim0gMPlQ,3,2008-11-23,another pie place that I would be happy to mee...,0,0,0,another pie place that I would be happy to mee...
1,BkiZn5XSzAv9q7J7_4HP7A,7HFkF9A2xaHxLqwcFnJ0JQ,N_2yEZ41g9zDW_gWArFiHw,4,2017-03-30,I came with my sister and was very excited fro...,1,0,1,I came with my sister and was very excited fro...
2,L6kc7Nr7hWiqo7ZvWu4rtg,rsi9Sf_Efk55hPRk6k7vpQ,XhLM_OtYslzyd4GyvpqACA,1,2017-03-19,I am very disappointed with the Brazilian I re...,0,0,0,I am very disappointed with the Brazilian I re...
3,y35xKzutHXT985mUpp5qVA,fKSX1CAoXkyQecAWR02DVQ,ILa-Xv5-h23A9OMrYK17oQ,4,2017-05-04,Stopped for lunch with my wife. I had the Mont...,0,0,1,Stopped for lunch with my wife I had the Monte...
4,UqQGtBDEfkYMLV-FykNCdA,YmTYfzrC4Z2JbyOucGVJtA,gBfPyzPRmeOaj3SdcIj0Rw,1,2010-05-20,"DON'T DO IT! You're in Vegas, looking for a d...",1,0,0,DONT DO IT Youre in Vegas looking for a dinin...


In [6]:
# Tokenization
import re

# Function to Tokenize words
def tokenize(text):

  # W+ means that either a word character (A-Za-z0-9) or a dash (-) can go there
  tokens = re.split('\W+', text)
  return tokens

# Convert to lowercase as Python is case-sensitive
pandas_df['body_text_tokenized'] = pandas_df['body_text_clean'].apply(lambda x: tokenize(x.lower()))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized
0,K8avYPWsh45v7VoZgQz-Vg,mrEoKZsYtSgMJsIuLr_oTg,UGyEr_PMA-v1cuim0gMPlQ,3,2008-11-23,another pie place that I would be happy to mee...,0,0,0,another pie place that I would be happy to mee...,"[another, pie, place, that, i, would, be, happ..."
1,BkiZn5XSzAv9q7J7_4HP7A,7HFkF9A2xaHxLqwcFnJ0JQ,N_2yEZ41g9zDW_gWArFiHw,4,2017-03-30,I came with my sister and was very excited fro...,1,0,1,I came with my sister and was very excited fro...,"[i, came, with, my, sister, and, was, very, ex..."
2,L6kc7Nr7hWiqo7ZvWu4rtg,rsi9Sf_Efk55hPRk6k7vpQ,XhLM_OtYslzyd4GyvpqACA,1,2017-03-19,I am very disappointed with the Brazilian I re...,0,0,0,I am very disappointed with the Brazilian I re...,"[i, am, very, disappointed, with, the, brazili..."
3,y35xKzutHXT985mUpp5qVA,fKSX1CAoXkyQecAWR02DVQ,ILa-Xv5-h23A9OMrYK17oQ,4,2017-05-04,Stopped for lunch with my wife. I had the Mont...,0,0,1,Stopped for lunch with my wife I had the Monte...,"[stopped, for, lunch, with, my, wife, i, had, ..."
4,UqQGtBDEfkYMLV-FykNCdA,YmTYfzrC4Z2JbyOucGVJtA,gBfPyzPRmeOaj3SdcIj0Rw,1,2010-05-20,"DON'T DO IT! You're in Vegas, looking for a d...",1,0,0,DONT DO IT Youre in Vegas looking for a dinin...,"[dont, do, it, youre, in, vegas, looking, for,..."


In [7]:
# Remove all English stopwords
import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Function to remove stopwords
def remove_stopwords(tokenized_list):

  # Remove all stopwords
  text = [word for word in tokenized_list if word not in stopword]
  return text

pandas_df['body_text_nostop'] = pandas_df['body_text_tokenized'].apply(lambda x: remove_stopwords(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop
0,K8avYPWsh45v7VoZgQz-Vg,mrEoKZsYtSgMJsIuLr_oTg,UGyEr_PMA-v1cuim0gMPlQ,3,2008-11-23,another pie place that I would be happy to mee...,0,0,0,another pie place that I would be happy to mee...,"[another, pie, place, that, i, would, be, happ...","[another, pie, place, would, happy, meet, good..."
1,BkiZn5XSzAv9q7J7_4HP7A,7HFkF9A2xaHxLqwcFnJ0JQ,N_2yEZ41g9zDW_gWArFiHw,4,2017-03-30,I came with my sister and was very excited fro...,1,0,1,I came with my sister and was very excited fro...,"[i, came, with, my, sister, and, was, very, ex...","[came, sister, excited, hype, social, media, g..."
2,L6kc7Nr7hWiqo7ZvWu4rtg,rsi9Sf_Efk55hPRk6k7vpQ,XhLM_OtYslzyd4GyvpqACA,1,2017-03-19,I am very disappointed with the Brazilian I re...,0,0,0,I am very disappointed with the Brazilian I re...,"[i, am, very, disappointed, with, the, brazili...","[disappointed, brazilian, received, today, got..."
3,y35xKzutHXT985mUpp5qVA,fKSX1CAoXkyQecAWR02DVQ,ILa-Xv5-h23A9OMrYK17oQ,4,2017-05-04,Stopped for lunch with my wife. I had the Mont...,0,0,1,Stopped for lunch with my wife I had the Monte...,"[stopped, for, lunch, with, my, wife, i, had, ...","[stopped, lunch, wife, monte, cristo, potato, ..."
4,UqQGtBDEfkYMLV-FykNCdA,YmTYfzrC4Z2JbyOucGVJtA,gBfPyzPRmeOaj3SdcIj0Rw,1,2010-05-20,"DON'T DO IT! You're in Vegas, looking for a d...",1,0,0,DONT DO IT Youre in Vegas looking for a dinin...,"[dont, do, it, youre, in, vegas, looking, for,...","[dont, youre, vegas, looking, dining, hot, spo..."


In [9]:
# Stemming
from nltk.stem import PorterStemmer

# Create an instance for stemmer
ps = nltk.PorterStemmer()

# Function for stemming
def stemming(tokenized_text):

  text = [ps.stem(word) for word in tokenized_text]
  return text

pandas_df['body_text_stemmed'] = pandas_df['body_text_nostop'].apply(lambda x: stemming(x))

pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed
0,K8avYPWsh45v7VoZgQz-Vg,mrEoKZsYtSgMJsIuLr_oTg,UGyEr_PMA-v1cuim0gMPlQ,3,2008-11-23,another pie place that I would be happy to mee...,0,0,0,another pie place that I would be happy to mee...,"[another, pie, place, that, i, would, be, happ...","[another, pie, place, would, happy, meet, good...","[anoth, pie, place, would, happi, meet, good, ..."
1,BkiZn5XSzAv9q7J7_4HP7A,7HFkF9A2xaHxLqwcFnJ0JQ,N_2yEZ41g9zDW_gWArFiHw,4,2017-03-30,I came with my sister and was very excited fro...,1,0,1,I came with my sister and was very excited fro...,"[i, came, with, my, sister, and, was, very, ex...","[came, sister, excited, hype, social, media, g...","[came, sister, excit, hype, social, media, got..."
2,L6kc7Nr7hWiqo7ZvWu4rtg,rsi9Sf_Efk55hPRk6k7vpQ,XhLM_OtYslzyd4GyvpqACA,1,2017-03-19,I am very disappointed with the Brazilian I re...,0,0,0,I am very disappointed with the Brazilian I re...,"[i, am, very, disappointed, with, the, brazili...","[disappointed, brazilian, received, today, got...","[disappoint, brazilian, receiv, today, got, ho..."
3,y35xKzutHXT985mUpp5qVA,fKSX1CAoXkyQecAWR02DVQ,ILa-Xv5-h23A9OMrYK17oQ,4,2017-05-04,Stopped for lunch with my wife. I had the Mont...,0,0,1,Stopped for lunch with my wife I had the Monte...,"[stopped, for, lunch, with, my, wife, i, had, ...","[stopped, lunch, wife, monte, cristo, potato, ...","[stop, lunch, wife, mont, cristo, potato, sala..."
4,UqQGtBDEfkYMLV-FykNCdA,YmTYfzrC4Z2JbyOucGVJtA,gBfPyzPRmeOaj3SdcIj0Rw,1,2010-05-20,"DON'T DO IT! You're in Vegas, looking for a d...",1,0,0,DONT DO IT Youre in Vegas looking for a dinin...,"[dont, do, it, youre, in, vegas, looking, for,...","[dont, youre, vegas, looking, dining, hot, spo...","[dont, your, vega, look, dine, hot, spot, your..."


In [10]:
# Lemmatization
# import these modules 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 

# Create an instance
wn = nltk.WordNetLemmatizer()

def lemmatizing(tokenized_text):

  text = [wn.lemmatize(word) for word in tokenized_text]
  return text

pandas_df['body_text_lemmatized'] = pandas_df['body_text_nostop'].apply(lambda x: lemmatizing(x))

pandas_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized
0,K8avYPWsh45v7VoZgQz-Vg,mrEoKZsYtSgMJsIuLr_oTg,UGyEr_PMA-v1cuim0gMPlQ,3,2008-11-23,another pie place that I would be happy to mee...,0,0,0,another pie place that I would be happy to mee...,"[another, pie, place, that, i, would, be, happ...","[another, pie, place, would, happy, meet, good...","[anoth, pie, place, would, happi, meet, good, ...","[another, pie, place, would, happy, meet, good..."
1,BkiZn5XSzAv9q7J7_4HP7A,7HFkF9A2xaHxLqwcFnJ0JQ,N_2yEZ41g9zDW_gWArFiHw,4,2017-03-30,I came with my sister and was very excited fro...,1,0,1,I came with my sister and was very excited fro...,"[i, came, with, my, sister, and, was, very, ex...","[came, sister, excited, hype, social, media, g...","[came, sister, excit, hype, social, media, got...","[came, sister, excited, hype, social, medium, ..."
2,L6kc7Nr7hWiqo7ZvWu4rtg,rsi9Sf_Efk55hPRk6k7vpQ,XhLM_OtYslzyd4GyvpqACA,1,2017-03-19,I am very disappointed with the Brazilian I re...,0,0,0,I am very disappointed with the Brazilian I re...,"[i, am, very, disappointed, with, the, brazili...","[disappointed, brazilian, received, today, got...","[disappoint, brazilian, receiv, today, got, ho...","[disappointed, brazilian, received, today, got..."
3,y35xKzutHXT985mUpp5qVA,fKSX1CAoXkyQecAWR02DVQ,ILa-Xv5-h23A9OMrYK17oQ,4,2017-05-04,Stopped for lunch with my wife. I had the Mont...,0,0,1,Stopped for lunch with my wife I had the Monte...,"[stopped, for, lunch, with, my, wife, i, had, ...","[stopped, lunch, wife, monte, cristo, potato, ...","[stop, lunch, wife, mont, cristo, potato, sala...","[stopped, lunch, wife, monte, cristo, potato, ..."
4,UqQGtBDEfkYMLV-FykNCdA,YmTYfzrC4Z2JbyOucGVJtA,gBfPyzPRmeOaj3SdcIj0Rw,1,2010-05-20,"DON'T DO IT! You're in Vegas, looking for a d...",1,0,0,DONT DO IT Youre in Vegas looking for a dinin...,"[dont, do, it, youre, in, vegas, looking, for,...","[dont, youre, vegas, looking, dining, hot, spo...","[dont, your, vega, look, dine, hot, spot, your...","[dont, youre, vega, looking, dining, hot, spot..."


In [11]:
pandas_df['length'] = pandas_df['text'].apply(len)
pandas_df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool,body_text_clean,body_text_tokenized,body_text_nostop,body_text_stemmed,body_text_lemmatized,length
0,K8avYPWsh45v7VoZgQz-Vg,mrEoKZsYtSgMJsIuLr_oTg,UGyEr_PMA-v1cuim0gMPlQ,3,2008-11-23,another pie place that I would be happy to mee...,0,0,0,another pie place that I would be happy to mee...,"[another, pie, place, that, i, would, be, happ...","[another, pie, place, would, happy, meet, good...","[anoth, pie, place, would, happi, meet, good, ...","[another, pie, place, would, happy, meet, good...",256
1,BkiZn5XSzAv9q7J7_4HP7A,7HFkF9A2xaHxLqwcFnJ0JQ,N_2yEZ41g9zDW_gWArFiHw,4,2017-03-30,I came with my sister and was very excited fro...,1,0,1,I came with my sister and was very excited fro...,"[i, came, with, my, sister, and, was, very, ex...","[came, sister, excited, hype, social, media, g...","[came, sister, excit, hype, social, media, got...","[came, sister, excited, hype, social, medium, ...",480
2,L6kc7Nr7hWiqo7ZvWu4rtg,rsi9Sf_Efk55hPRk6k7vpQ,XhLM_OtYslzyd4GyvpqACA,1,2017-03-19,I am very disappointed with the Brazilian I re...,0,0,0,I am very disappointed with the Brazilian I re...,"[i, am, very, disappointed, with, the, brazili...","[disappointed, brazilian, received, today, got...","[disappoint, brazilian, receiv, today, got, ho...","[disappointed, brazilian, received, today, got...",236
3,y35xKzutHXT985mUpp5qVA,fKSX1CAoXkyQecAWR02DVQ,ILa-Xv5-h23A9OMrYK17oQ,4,2017-05-04,Stopped for lunch with my wife. I had the Mont...,0,0,1,Stopped for lunch with my wife I had the Monte...,"[stopped, for, lunch, with, my, wife, i, had, ...","[stopped, lunch, wife, monte, cristo, potato, ...","[stop, lunch, wife, mont, cristo, potato, sala...","[stopped, lunch, wife, monte, cristo, potato, ...",265
4,UqQGtBDEfkYMLV-FykNCdA,YmTYfzrC4Z2JbyOucGVJtA,gBfPyzPRmeOaj3SdcIj0Rw,1,2010-05-20,"DON'T DO IT! You're in Vegas, looking for a d...",1,0,0,DONT DO IT Youre in Vegas looking for a dinin...,"[dont, do, it, youre, in, vegas, looking, for,...","[dont, youre, vegas, looking, dining, hot, spo...","[dont, your, vega, look, dine, hot, spot, your...","[dont, youre, vega, looking, dining, hot, spot...",3123


In [11]:
# Apply CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(pandas_df['review_text'])

KeyError: ignored

In [ ]:
# import pandas as pd

# X_counts_df = pd.DataFrame(X_counts.toarray(), columns=count_vect.get_feature_names())
# X_counts_df.head(10)

In [50]:
pandas_df_copy = pandas_df.copy()

pandas_df_copy = pandas_df_copy[['review_id', 'text', 'stars', 'cool', 'useful', 'funny', 'date', 'business_id', 'user_id', 'length', 'body_text_nostop', 'body_text_stemmed', 'body_text_lemmatized']]

# Convert pandas_df to sparks df
spark_df = spark.createDataFrame(pandas_df_copy)
spark_df.show(5)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|K8avYPWsh45v7VoZg...|another pie place...|    3|   0|     0|    0|2008-11-23|UGyEr_PMA-v1cuim0...|mrEoKZsYtSgMJsIuL...|   256|[another, pie, pl...|[anoth, pie, plac...|[another, pie, pl...|
|BkiZn5XSzAv9q7J7_...|I came with my si...|    4|   1|     1|    0|2017-03-30|N_2yEZ41g9zDW_gWA...|7HFkF9A2xaHxLqwcF...|   480|[came, sister, ex...|[came, sister, ex...|[came, sister, ex...|
|L6kc7Nr7hWiqo7ZvW...|I am very disappo...|  

In [51]:
# Import functions
from pyspark.ml.feature import HashingTF, IDF, StringIndexer

In [52]:
# Make stars values a list
from pyspark.sql.functions import col, split
spark_df = spark_df.withColumn("star_array", split(col("stars"), " "))
spark_df.show()

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+
|K8avYPWsh45v7VoZg...|another pie place...|    3|   0|     0|    0|2008-11-23|UGyEr_PMA-v1cuim0...|mrEoKZsYtSgMJsIuL...|   256|[another, pie, pl...|[anoth, pie, plac...|[another, pie, pl...|       [3]|
|BkiZn5XSzAv9q7J7_...|I came with my si...|    4|   1|     1|    0|2017-03-30|N_2yEZ41g9zDW_gWA...|7HFkF9A2xaHxLqwcF...|   480|[came, sister, ex...|[came, sister, ex...|[came, sister, ex...|  

In [53]:
# Initialize a CoutVectorizer
from pyspark.ml.feature import CountVectorizer
star_vectorizer = CountVectorizer(inputCol="star_array", outputCol="stars_one_hot", vocabSize=5, minDF=1.0)

In [54]:
# Create a vector model
star_vector_model = star_vectorizer.fit(spark_df)

In [55]:
# One hot encoded column
df_ohe = star_vector_model.transform(spark_df)
df_ohe.show(3)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|stars_one_hot|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+
|K8avYPWsh45v7VoZg...|another pie place...|    3|   0|     0|    0|2008-11-23|UGyEr_PMA-v1cuim0...|mrEoKZsYtSgMJsIuL...|   256|[another, pie, pl...|[anoth, pie, plac...|[another, pie, pl...|       [3]|(5,[2],[1.0])|
|BkiZn5XSzAv9q7J7_...|I came with my si...|    4|   1|     1|    0|2017-03-30|N_2yEZ41g9zDW_gWA...|7HFkF9A2xaHxLqwcF...|   480|[came, si

In [56]:
# Create all the features to the data set
star_rating = StringIndexer(inputCol='stars',outputCol='label')
hashingTF = HashingTF(inputCol="body_text_stemmed", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [57]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
# Create feature vector 
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [58]:
# Create and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[star_rating, hashingTF, idf, clean_up])

In [59]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df_ohe)
cleaned = cleaner.transform(df_ohe)

In [60]:
cleaned.show(5)

+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+-----+--------------------+--------------------+--------------------+
|           review_id|                text|stars|cool|useful|funny|      date|         business_id|             user_id|length|    body_text_nostop|   body_text_stemmed|body_text_lemmatized|star_array|stars_one_hot|label|          hash_token|           idf_token|            features|
+--------------------+--------------------+-----+----+------+-----+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+----------+-------------+-----+--------------------+--------------------+--------------------+
|K8avYPWsh45v7VoZg...|another pie place...|    3|   0|     0|    0|2008-11-23|UGyEr_PMA-v1cuim0...|mrEoKZsYtSgMJsIuL...|   256|[another, pie, pl.

<br></br>**Pipeline**<br></br>

In [ ]:
# Import functions
# from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [ ]:
# Make stars values a list
# from pyspark.sql.functions import col, split
# spark_df = spark_df.withColumn("star_array", split(col("stars"), " "))
# spark_df.show()

In [ ]:
# Initialize a CoutVectorizer
# from pyspark.ml.feature import CountVectorizer
# star_vectorizer = CountVectorizer(inputCol="star_array", outputCol="stars_one_hot", vocabSize=5, minDF=1.0)

In [ ]:
# Create a vector model
# star_vector_model = star_vectorizer.fit(spark_df)

In [ ]:
# # One hot encoded column
# df_ohe = star_vector_model.transform(spark_df)
# df_ohe.show(3)

In [ ]:
# Create all the features to the data set
#star_rating = StringIndexer(inputCol='stars_one_hot',outputCol='label')
# tokenizer = Tokenizer(inputCol="review_text", outputCol="token_text")
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
# hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
# idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.linalg import Vector
# # Create feature vector 
# clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create and run a data processing Pipeline
# from pyspark.ml import Pipeline
# data_prep_pipeline = Pipeline(stages=[tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
# Fit and transform the pipeline
# cleaner = data_prep_pipeline.fit(df_ohe)
# cleaned = cleaner.transform(df_ohe)

In [ ]:
# cleaned.show()

In [ ]:
# Drop redundant column
# x=cleaned.drop('review_type')

# **Naive Bayes**

In [61]:
#Drop intermediate columns
x=cleaned.select('features', 'label')
x.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[22567,27...|  2.0|
|(262145,[427,1253...|  0.0|
|(262145,[17893,28...|  3.0|
|(262145,[31463,65...|  0.0|
|(262145,[2396,392...|  3.0|
+--------------------+-----+
only showing top 5 rows



In [24]:
x.dtypes

[('features', 'vector'), ('label', 'double')]

In [25]:
# Import col
from pyspark.sql.functions import col

# Change column DataType for stars
x = x.withColumn('label', col('label').cast('int'))

x.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[22567,27...|    2|
|(262145,[427,1253...|    0|
|(262145,[17893,28...|    3|
|(262145,[31463,65...|    0|
|(262145,[2396,392...|    3|
+--------------------+-----+
only showing top 5 rows



In [ ]:
# Rename class to label
# x = x.withColumnRenamed('stars_one_hot', 'label')
# x.show(5)

In [ ]:
#import pandas as pd
# data_df = x.select('*').toPandas()
# x = spark.createDataFrame(data_df)

In [28]:
# Break data down into a training set and a testing set
training, testing = x.randomSplit([0.7, 0.3], 21)

**Naive Bayes**

In [29]:
from pyspark.ml.classification import NaiveBayes
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [30]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.358936


Accuracy of model at predicting reviews was: 0.522249

**Logistic Regression**

In [29]:
from pyspark.ml.classification import LogisticRegression
# Create a Naive Bayes model and fit training data
lg = LogisticRegression()
predictor = lg.fit(training)

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(predictor.transform(testing))
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.444839


Accuracy of model at predicting reviews was: 0.701011

**Multilayer Percepron**

In [62]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [63]:
# Split the data into train and test
splits = x.randomSplit([0.8, 0.2], 1234)
train = splits[0]
test = splits[1]

In [64]:
train.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(262145,[15,427,2...|  3.0|
|(262145,[15,991,2...|  0.0|
|(262145,[15,1353,...|  2.0|
|(262145,[15,4914,...|  1.0|
|(262145,[15,5381,...|  1.0|
|(262145,[29,1353,...|  0.0|
|(262145,[47,1353,...|  2.0|
|(262145,[47,2325,...|  1.0|
|(262145,[47,12325...|  1.0|
|(262145,[47,34343...|  0.0|
|(262145,[68,5173,...|  0.0|
|(262145,[70,3856,...|  2.0|
|(262145,[74,3067,...|  2.0|
|(262145,[91,991,4...|  0.0|
|(262145,[97,440,9...|  4.0|
|(262145,[97,635,9...|  3.0|
|(262145,[97,976,3...|  2.0|
|(262145,[97,1353,...|  4.0|
|(262145,[97,1353,...|  4.0|
|(262145,[97,1353,...|  4.0|
+--------------------+-----+
only showing top 20 rows



In [69]:
# specify layers for the neural network:
# input layer of size 3 (features), two intermediate of size 5 and 4
# and output of size 5 (classes)
layers = [262145, 256, 5]

In [70]:
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: ignored

In [72]:
# train the model
model = trainer.fit(train)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45873)
Traceback (most recent call last):
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45873)
Traceback (most recent call last):
  File "/content/spark-2.4.6-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

Dur

Py4JNetworkError: ignored

In [ ]:
# Authenticate user
from google.colab import auth
auth.authenticate_user()

In [ ]:
# Set project id
project_id = 'xy-yelp'

In [ ]:
# Set project
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
# File path to save json file to
filepath = '/tmp/ml_j.json'

In [ ]:
# Save json file **** this will break if the file already exists, which it does at this point, therefore its commented out for now
x.coalesce(1).write.format('json').save(filepath)

In [ ]:
# Bucket name
bucket_name = 'xy-bucket'

In [ ]:
# Copy saved file from /tmp to bucket
!gsutil cp -r /tmp/ml_j.json/ gs://{bucket_name}/json_files/

Copying file:///tmp/ml_j.json/.part-00000-f8a8f21c-0ebe-434c-a8b3-a5c8988dd298-c000.json.crc [Content-Type=application/octet-stream]...
Copying file:///tmp/ml_j.json/._SUCCESS.crc [Content-Type=application/octet-stream]...
Copying file:///tmp/ml_j.json/_SUCCESS [Content-Type=application/octet-stream]...
Copying file:///tmp/ml_j.json/part-00000-f8a8f21c-0ebe-434c-a8b3-a5c8988dd298-c000.json [Content-Type=application/json]...
\
Operation completed over 4 objects/63.0 MiB.                                     
